# Import Libraries

In [97]:
# QT backend
%matplotlib qt

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
import copy

# Toggle fullscreen mode for all plots
plt.rcParams['figure.figsize'] = [12, 8]

# Read Data

In [98]:
points = pd.read_excel('15-Points.xlsx')
points.head()

,x,y,City
0,5.500000e-08,9.860000e-09,1
1,-2.887330e+01,-7.980000e-08,2
2,-7.929160e+01,-2.140330e+01,3
3,-1.465770e+01,-4.338960e+01,4
4,-6.474730e+01,2.189820e+01,5


# Genetic Algorithm

In [99]:
class Chromosome:
    def __init__(self, genes=None):
        self.genes = genes
    
    @property
    def genes(self):
        return self.__genes
    
    @property
    def fitness(self):
        return self.__fitness
    
    @property
    def cost(self):
        return self.__cost

    @genes.setter
    def genes(self, genes):
        
        if genes:
            self.__genes = genes
        else:
            self.__genes = list(np.random.permutation(15))
            self.__genes += [self.__genes[0]]
        
        self.cost, self.fitness = self.__get_fitness()

    @fitness.setter
    def fitness(self, fitness):
        self.__fitness = fitness

    @cost.setter
    def cost(self, cost):
        self.__cost = cost
    
    # Problem Dependant
    def __get_fitness(self):
        cities_position = points.values[self.genes][:, :2]
        cost = sum(np.linalg.norm(city1 - city2) for city1, city2 in zip(cities_position, cities_position[1:]))
        return cost, 1/cost


    @staticmethod
    def crossover_utility(chromosome_1, chromosome_2):
        # Merge two subsets with duplicates
        crossover_point = np.random.randint(1, 15)
        sub_1 = chromosome_1.genes[:crossover_point]
        sub_2 = chromosome_2.genes[crossover_point:-1]
        child_genes = sub_1 + sub_2
            
        # substitute duplicates with values don't exist before
        substitutions = list(set(range(15)) - set(child_genes))
        
        child_genes = child_genes[:len(sub_1)] + \
            [substitutions.pop(0) if g in sub_1 else g for g in sub_2] + \
              [child_genes[0]]
        
        return Chromosome(genes=child_genes)


    @staticmethod
    def crossover(chromosome_1, chromosome_2):
        offspring_1 = Chromosome.crossover_utility(chromosome_1, chromosome_2)
        offspring_2 = Chromosome.crossover_utility(chromosome_2, chromosome_1)

        return offspring_1, offspring_2
    
    def swap_mutation(self):
        # Generate two unique random numbers in the range [1, 14]
        indices = np.random.choice(range(1, 15), size=2, replace=False)

        # Swaping
        self.genes[indices[0]], self.genes[indices[1]] = self.genes[indices[1]], self.genes[indices[0]]
        self.cost, self.fitness = self.__get_fitness()

    def __repr__(self) -> str:
        info = {
                'genes': self.genes,
                'cost': self.cost,
                'fitness': self.fitness
                }
                
        return f'{info}'

In [100]:
class GeneticAlgorithm:
    def __init__(self, population_size, crossover_rate, mutation_rate, elite_size, max_generations):
        self.population_size = population_size
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.elite_size = elite_size
        self.max_generations = max_generations
        self.population = []
        self.best_chromosome = None

    def initialize_population(self):
        self.population = [Chromosome() for _ in range(self.population_size)]
        self.best_chromosome = max(self.population, key=lambda x: x.fitness)

    def selection(self):
        fitness_values = [chromosome.fitness for chromosome in self.population]
        probabilities = np.array(fitness_values) / sum(fitness_values)
        selected_indices = np.random.choice(range(self.population_size), size=self.population_size, p=probabilities)
        selected_population = [self.population[i] for i in selected_indices]
        elite_population = sorted(selected_population, key=lambda x: x.fitness, reverse=True)[:self.elite_size]
        return elite_population

    def crossover(self, parent_1, parent_2):
        if np.random.rand() < self.crossover_rate:
            offspring_1, offspring_2 = Chromosome.crossover(parent_1, parent_2)
            return offspring_1, offspring_2
        
        return parent_1, parent_2

    def mutation(self, chromosome):
        if np.random.rand() < self.mutation_rate:
            chromosome.swap_mutation()
        
        return chromosome

    def evolve(self):
        self.initialize_population()
        
        for generation in range(self.max_generations):
            selected_population = self.selection()
            offspring_population = []

            for i in range(self.population_size - self.elite_size):
                parent_1, parent_2 = np.random.choice(selected_population, size=2, replace=False)
                offspring_1, offspring_2 = self.crossover(parent_1, parent_2)
                offspring_1 = self.mutation(offspring_1)
                offspring_2 = self.mutation(offspring_2)
                offspring_population.extend([offspring_1, offspring_2])

            offspring_population = sorted(offspring_population, key=lambda x: x.fitness, reverse=True)[:self.population_size - self.elite_size]
            self.population = selected_population + offspring_population

            # Compare between best chromosome in current generation and best chromose ever
            generation_best_chromosome = copy.deepcopy(max(self.population, key=lambda x: x.fitness))
            best_chromosomes = [self.best_chromosome, generation_best_chromosome]
            self.best_chromosome = copy.deepcopy(max(best_chromosomes, key=lambda x: x.fitness))

            
            # print best chromosome in current generation
            print(f"Generation: {generation}, Best Chromosome: {generation_best_chromosome}")

            # visualize best chromosome in current generation
            path = generation_best_chromosome.genes

            Xs = points.values[path][:, 0]
            Ys = points.values[path][:, 1]
            cities_name = list(map(lambda x: 'City '+x, list((points.values[path][:, 2]).astype('int').astype('str'))))

            interactive_figure = plt.plot(points.values[path][:, 0], points.values[path][:, 1], marker='o', color='b')          
            
            texts = []
            for i in range(len(Xs[:-1])):
                texts.append(plt.text(Xs[i], Ys[i]+2, f'{cities_name[i]}', ha='center'))  

            plt.title("Generation: {}, Best Cost: {:.2f}".format(generation, generation_best_chromosome.cost))
            plt.pause(0.1)

            for i in range(len(Xs[:-1])):
                texts[i].remove()
            
            plt.setp(interactive_figure, xdata=[], ydata=[])

        # print best chromosome ever
        print(f"Best Chromosome Ever: {self.best_chromosome}")

        # visualize best chromosome in current generation
        path = generation_best_chromosome.genes

        Xs = points.values[path][:, 0]
        Ys = points.values[path][:, 1]
        cities_name = list(map(lambda x: 'City '+x, list((points.values[path][:, 2]).astype('int').astype('str'))))

        interactive_figure = plt.plot(points.values[path][:, 0], points.values[path][:, 1], marker='o', color='b')          
        
        texts = []
        for i in range(len(Xs[:-1])):
            texts.append(plt.text(Xs[i], Ys[i]+2, f'{cities_name[i]}', ha='center'))  

        plt.title("Generation: {}, Best Cost: {:.2f}".format(generation, generation_best_chromosome.cost))
        plt.pause(2)
        plt.close()
                
        return self.best_chromosome

In [101]:
ga = GeneticAlgorithm(population_size=100, crossover_rate=0.8, mutation_rate=0.2, elite_size=10, max_generations=100)
best_chromosome = ga.evolve()

Generation: 0, Best Chromosome: {'genes': [6, 2, 11, 13, 3, 0, 1, 5, 7, 9, 4, 12, 10, 14, 8, 6], 'cost': 478.9499115070315, 'fitness': 0.002087901001700715}
Generation: 1, Best Chromosome: {'genes': [6, 2, 11, 13, 3, 0, 1, 5, 7, 9, 4, 12, 10, 14, 8, 6], 'cost': 478.9499115070315, 'fitness': 0.002087901001700715}
Generation: 2, Best Chromosome: {'genes': [6, 2, 11, 13, 9, 0, 10, 3, 7, 5, 1, 12, 4, 14, 8, 6], 'cost': 389.13338952152935, 'fitness': 0.0025698128891729903}
Generation: 3, Best Chromosome: {'genes': [6, 2, 11, 13, 9, 0, 10, 3, 7, 5, 1, 12, 4, 14, 8, 6], 'cost': 389.13338952152935, 'fitness': 0.0025698128891729903}
Generation: 4, Best Chromosome: {'genes': [6, 2, 11, 13, 9, 0, 10, 3, 7, 5, 1, 12, 4, 14, 8, 6], 'cost': 389.13338952152935, 'fitness': 0.0025698128891729903}
Generation: 5, Best Chromosome: {'genes': [6, 2, 11, 13, 9, 0, 10, 3, 7, 5, 1, 12, 8, 14, 4, 6], 'cost': 372.6325834349943, 'fitness': 0.0026836085851157186}
Generation: 6, Best Chromosome: {'genes': [6, 2, 11

In [102]:
path = best_chromosome.genes

cities_name = list(map(lambda x: 'City '+x, list((points.values[path][:, 2]).astype('int').astype('str'))))
fig = px.line(x=points.values[path][:, 0], y=points.values[path][:, 1], hover_name=cities_name, markers=True, text=cities_name)
fig.update_traces(textposition='top center')

# Set the title of the chart
fig.update_layout(title='Travel Salesman')

# Show the chart
fig.show()